# Julia Workshop: Optimization and Solvers

## @ CEF 2017

**Authors**: Chase Coleman and Spencer Lyon

**Date**: 27 June 2017


## Goal

A few different packages used for optimization or non-linear equation solving.


## QuantEcon

The QuantEcon library has a few simple optimizers and solvers. See the economics [notebook](economics.ipynb) for more information.

In [1]:
# Pkg.add("QuantEcon")

## Optim.jl

Package for optimizing written in pure Julia

In [2]:
# Pkg.add("Optim")

In [3]:
using Optim

In [4]:
rosenbrock(x) = (1.0 - x[1])^2 + 100.0*(x[2] - x[1]^2)^2

rosenbrock (generic function with 1 method)

### Optimizing without gradient

In [6]:
optimize(rosenbrock, zeros(2), NelderMead())

LoadError: error compiling initial_state: error compiling centroid!: error compiling scale!: could not load library "libopenblas64_"
libgfortran.so.3: cannot open shared object file: No such file or directory

In [7]:
optimize(rosenbrock, zeros(2), BFGS())

LoadError: error compiling optimize: error compiling optimize: error compiling vecnorm: error compiling vecnorm2: could not load library "libopenblas64_"
libgfortran.so.3: cannot open shared object file: No such file or directory

### Optimizing with gradient

In [8]:
function rosenbrock_grad!(x::Vector, grad::Vector)
    grad[1] = -2.0*(1.0 - x[1]) - 400.0*(x[2] - x[1]^2)*x[1]
    grad[2] = 200.0*(x[2] - x[1]^2)
end

rosenbrock_grad! (generic function with 1 method)

In [9]:
optimize(rosenbrock, rosenbrock_grad!, zeros(2), LBFGS(),
         Optim.Options(x_tol=1e-10, f_tol=1e-9, iterations=25000,
                       allow_f_increases=true))

LoadError: error compiling optimize: error compiling optimize: error compiling vecnorm: error compiling vecnorm2: could not load library "libopenblas64_"
libgfortran.so.3: cannot open shared object file: No such file or directory

In [10]:
optimize(rosenbrock, rosenbrock_grad!, zeros(2), GradientDescent(),
         Optim.Options(x_tol=1e-10, f_tol=1e-9, iterations=25000,
                       allow_f_increases=true))

LoadError: error compiling optimize: error compiling optimize: error compiling vecnorm: error compiling vecnorm2: could not load library "libopenblas64_"
libgfortran.so.3: cannot open shared object file: No such file or directory

### Optimizing with Hessian

In [11]:
function rosenbrock_hess!(x::Vector, hess::Matrix)
    hess[1, 1] = 2.0 - 400.0 * x[2] + 1200.0*x[1]^2
    hess[1, 2] = -400.0 * x[1]
    hess[2, 1] = -400.0 * x[1]
    hess[2, 2] = 200.0
end

rosenbrock_hess! (generic function with 1 method)

In [12]:
optimize(rosenbrock, rosenbrock_grad!, rosenbrock_hess!, zeros(2), Newton())

LoadError: error compiling optimize: error compiling optimize: error compiling vecnorm: error compiling vecnorm2: could not load library "libopenblas64_"
libgfortran.so.3: cannot open shared object file: No such file or directory

## NLopt

Julia wrapper to high quality C library.

This library has _lots_ of options for algorithms. See [list](http://ab-initio.mit.edu/wiki/index.php/NLopt_Algorithms) of algorithms

In [15]:
# Pkg.add("NLopt")

In [ ]:
using NLopt

In [ ]:
function rosenbrock(x::Vector, grad::Vector)
    if length(grad) > 0
        rosenbrock_grad!(x, grad)
    end

    return rosenbrock(x)
end

### A non-gradient and gradient based method

In [ ]:
opt_NM = Opt(:NLOPT_LN_NELDERMEAD, 2)

min_objective!(opt_NM, rosenbrock)

optimize(opt_NM, zeros(2))

In [ ]:
opt_MMA = Opt(:NLOPT_LD_MMA, 2)

min_objective!(opt_MMA, rosenbrock)

optimize(optMMA, zeros(2))

### A constrained optimization problem

\begin{align*}
  \min_{x_1, x_2} &\sqrt{x_2} \\
  &\text{subject to } \\
  &x_2 \geq 0 \\
  &x_2 \geq (2 x_1)^3  \\
  &x_2 \geq (-x_1 + 1)^3
\end{align*}

In [ ]:
function myfunc(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end

    return sqrt(x[1] + x[2])
end

function myconstraint(x::Vector, grad::Vector, a, b)
    if length(grad) > 0
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end

    return (a*x[1] + b)^3 - x[2]
end

opt = Opt(:LD_MMA, 2)

lower_bounds!(opt, [-Inf, 0.])
xtol_rel!(opt, 1e-6)

min_objective!(opt, myfunc)

inequality_constraint!(opt, (x,g) -> myconstraint(x, g, 2.0, 0.0), 1e-8)
inequality_constraint!(opt, (x,g) -> myconstraint(x, g, -1.0 ,1.0), 1e-8)

(minf, minx, ret) = optimize(opt, [1.234, 5.678])

println("got $minf at $minx (returned $ret)")

## NLsolve

Julia package written to solve systems of non-linear equations.

In [13]:
# Pkg.add("NLsolve")

In [14]:
using NLsolve

INFO: Recompiling stale cache file /home/chase/.julia/lib/v0.5/NLsolve.ji for module NLsolve.
ErrorException("could not load library "libopenblas64_"
libgfortran.so.3: cannot open shared object file: No such file or directory")
ErrorException("could not load library "libcholmod"
libgfortran.so.3: cannot open shared object file: No such file or directory")
ErrorException("could not load library "libopenblas64_"
libgfortran.so.3: cannot open shared object file: No such file or directory")
ErrorException("could not load library "libcholmod"
libgfortran.so.3: cannot open shared object file: No such file or directory")


In [ ]:
function f!(xy::Vector, fxy::Vector)
    # Pull out arguments
    x, y = xy

    # Fill fxy
    fxy[1] = x^2 - sin(y)
    fxy[2] = y^2 - cos(x)
end

function g!(xy::Vector, jacxy::Matrix)
    x, y = xy
    # Fill with derivatives of first function
    jacxy[1, 1] = 2*x
    jacxy[1, 2] = -cos(y)

    # Fill off-diagonal
    jacxy[2, 1] = sin(x)
    jacxy[2, 2] = 2*y
end

res = nlsolve(f!, g!, [0.0, 0.0], ftol=1e-10)